In [1]:
import random
import math
from itertools import permutations
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Model

In [3]:
all_rotation_orders = [''.join(axes) for axes in permutations({'x','y','z'})]
all_rotation_orders

['xzy', 'xyz', 'zxy', 'zyx', 'yxz', 'yzx']

In [4]:
def random_vector(n, mag):
    return 2.0 * mag * (0.5-np.random.random(size=n))
random_vector(10,5.0)

array([ 2.88906462,  1.61892164, -1.52130193, -0.69768431, -1.02284598,
        1.60113298, -2.04465857, -3.4423229 ,  4.65637925, -4.96397502])

define a function to generate a single sample from the model.

sample is a tuple of:
* np.array with 9 matrix elements + 3 rotation angles
* string corresponding to chosen rotation order (element of all_rotation_orders)

In [8]:
def generate_sample_from_model():
    rotation_order = random.choice(all_rotation_orders)  # choose a rotation order
    angles = random_vector(3, 10.0)   # generate three random angles (in degrees)
    if (min(abs(angles)) < 0.1):     # reject any angles that are too small
        return generate_sample_from_model()
    matrix = R.from_euler(rotation_order, angles, degrees = True)  # generate matrix
    noise_angles = angles + random_vector(3, 0.1)    
    x = np.concatenate((matrix.as_dcm().flatten(), 
                        np.array(noise_angles)))
                        # np.array(np.cos(noise_angles)), 
                        # np.array(np.sin(noise_angles))))                         
    return (x, rotation_order)
generate_sample_from_model()

(array([ 0.98515222, -0.03795229,  0.16743575,  0.03230152,  0.99881712,
         0.03634517, -0.16861708, -0.0303971 ,  0.98521282,  1.8100552 ,
        -2.15958383,  9.68338291]), 'zxy')

define a function to generate multiple samples, suitable for training or testing.

for number N, sample is a tuple:
 * np.array Nx12 of x vectors (standardized zero mean)
 * np.array Nx6 one-hot vectors

In [9]:
def generate_samples(n):
    [xs, rotation_orders] = list(zip(*[generate_sample_from_model() for _ in range(n)]))
    xs = np.array(xs)
    xs -= xs.mean(0)    # standardize along sample dimension
    xs /= xs.std(0)
    ys = [all_rotation_orders.index(rotation_order) for rotation_order in rotation_orders]
    return xs, keras.utils.to_categorical(ys)
generate_samples(3)

(array([[-1.12784362,  1.00513723,  1.2710693 , -1.0466738 , -0.97765887,
          0.77422513, -1.1065417 , -1.25275247, -1.26451946, -1.11083792,
          1.26069947, -0.87983508],
        [ 1.30282409,  0.35898485, -1.17245841, -0.30028648,  1.37377991,
         -1.41201821,  1.31594647,  1.19467056,  1.18066004, -0.20255817,
         -1.18530707,  1.39878024],
        [-0.17498046, -1.36412207, -0.0986109 ,  1.34696028, -0.39612104,
          0.63779308, -0.20940477,  0.05808191,  0.08385942,  1.31339609,
         -0.0753924 , -0.51894515]]), array([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]], dtype=float32))

generate training and testing data from the model

In [13]:
x_train, y_train = generate_samples(10000)
x_test, y_test = generate_samples(1000)
x_train.shape, y_train.shape

((10000, 12), (10000, 6))

now create an MLP as the model

In [14]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 4,166
Trainable params: 4,166
Non-trainable params: 0
_________________________________________________________________


set up the optimizer and compile the model

In [15]:
adam = Adam(lr=0.001, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

now fit the data using the training samples

In [16]:
model.fit(x_train, y_train, epochs=200, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
10000/10000 [==============================] - 0s 26us/step - loss: 1.7409 - acc: 0.2503
Epoch 2/500
10000/10000 [==============================] - 0s 5us/step - loss: 1.4878 - acc: 0.4019
Epoch 3/500
10000/10000 [==============================] - 0s 5us/step - loss: 1.1486 - acc: 0.5707
Epoch 4/500
10000/10000 [==============================] - 0s 6us/step - loss: 0.8402 - acc: 0.7200
Epoch 5/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.6206 - acc: 0.8036
Epoch 6/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.4826 - acc: 0.8527
Epoch 7/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.4071 - acc: 0.8700
Epoch 8/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.3467 - acc: 0.8970
Epoch 9/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.3022 - acc: 0.9164
Epoch 10/500
10000/10000 [=========================

10000/10000 [==============================] - 0s 5us/step - loss: 0.0644 - acc: 0.9815
Epoch 80/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0468 - acc: 0.9871
Epoch 81/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0460 - acc: 0.9861
Epoch 82/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0382 - acc: 0.9915
Epoch 83/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0380 - acc: 0.9908
Epoch 84/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0365 - acc: 0.9912
Epoch 85/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0361 - acc: 0.9917
Epoch 86/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0394 - acc: 0.9887
Epoch 87/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0363 - acc: 0.9915
Epoch 88/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0362 - acc: 0.9913
Epo

10000/10000 [==============================] - 0s 5us/step - loss: 0.0229 - acc: 0.9933
Epoch 160/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0216 - acc: 0.9943
Epoch 161/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0284 - acc: 0.9899
Epoch 162/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0209 - acc: 0.9943
Epoch 163/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0574 - acc: 0.9845
Epoch 164/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0229 - acc: 0.9933
Epoch 165/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0280 - acc: 0.9909
Epoch 166/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0263 - acc: 0.9915
Epoch 167/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0323 - acc: 0.9895
Epoch 168/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0221 - acc: 0

10000/10000 [==============================] - 0s 5us/step - loss: 0.0343 - acc: 0.9889
Epoch 240/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0227 - acc: 0.9920
Epoch 241/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0219 - acc: 0.9922
Epoch 242/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0202 - acc: 0.9935
Epoch 243/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0180 - acc: 0.9951
Epoch 244/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0136 - acc: 0.9970
Epoch 245/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0140 - acc: 0.9963
Epoch 246/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0144 - acc: 0.9964
Epoch 247/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0142 - acc: 0.9959
Epoch 248/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0128 - acc: 0

10000/10000 [==============================] - 0s 5us/step - loss: 0.0120 - acc: 0.9966
Epoch 320/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0151 - acc: 0.9948
Epoch 321/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0211 - acc: 0.9936
Epoch 322/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0175 - acc: 0.9940
Epoch 323/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0155 - acc: 0.9951
Epoch 324/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0221 - acc: 0.9921
Epoch 325/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0132 - acc: 0.9961
Epoch 326/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0138 - acc: 0.9955
Epoch 327/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0124 - acc: 0.9966
Epoch 328/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0157 - acc: 0

10000/10000 [==============================] - 0s 5us/step - loss: 0.0123 - acc: 0.9957
Epoch 400/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0105 - acc: 0.9969
Epoch 401/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0096 - acc: 0.9971
Epoch 402/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0102 - acc: 0.9970
Epoch 403/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0127 - acc: 0.9965
Epoch 404/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0104 - acc: 0.9966
Epoch 405/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0093 - acc: 0.9974
Epoch 406/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0087 - acc: 0.9968
Epoch 407/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0373 - acc: 0.9899
Epoch 408/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0233 - acc: 0

10000/10000 [==============================] - 0s 5us/step - loss: 0.0122 - acc: 0.9964
Epoch 480/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0110 - acc: 0.9970
Epoch 481/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0167 - acc: 0.9944
Epoch 482/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0355 - acc: 0.9910
Epoch 483/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0274 - acc: 0.9908
Epoch 484/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0202 - acc: 0.9933
Epoch 485/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0124 - acc: 0.9958
Epoch 486/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0104 - acc: 0.9970
Epoch 487/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0122 - acc: 0.9958
Epoch 488/500
10000/10000 [==============================] - 0s 5us/step - loss: 0.0706 - acc: 0

In [17]:
score = model.evaluate(x_test, y_test, batch_size=256)
print(score)

1000/1000 [==============================] - 0s 38us/step
[0.037424470499157904, 0.9839999966621399]


In [18]:
model.save('rotation_order.h5')

In [21]:
import h5py
f = h5py.File('rotation_order.h5')

In [22]:
f

<HDF5 file "rotation_order.h5" (mode r+)>

In [36]:
[(k, grp) for k in f.keys() for grp in f[k]]

[('model_weights', 'dense_1'),
 ('model_weights', 'dense_2'),
 ('model_weights', 'dense_3'),
 ('model_weights', 'dense_4'),
 ('optimizer_weights', 'Adam'),
 ('optimizer_weights', 'training')]

In [37]:
type(f)

h5py._hl.files.File

In [53]:
import pprint
f.visititems(lambda name, obj: pprint.pprint(obj))
# f['model_weights']['dense_1'].name

<HDF5 group "/model_weights" (4 members)>
<HDF5 group "/model_weights/dense_1" (1 members)>
<HDF5 group "/model_weights/dense_1/dense_1" (2 members)>
<HDF5 dataset "bias:0": shape (64,), type "<f4">
<HDF5 dataset "kernel:0": shape (12, 64), type "<f4">
<HDF5 group "/model_weights/dense_2" (1 members)>
<HDF5 group "/model_weights/dense_2/dense_2" (2 members)>
<HDF5 dataset "bias:0": shape (32,), type "<f4">
<HDF5 dataset "kernel:0": shape (64, 32), type "<f4">
<HDF5 group "/model_weights/dense_3" (1 members)>
<HDF5 group "/model_weights/dense_3/dense_3" (2 members)>
<HDF5 dataset "bias:0": shape (32,), type "<f4">
<HDF5 dataset "kernel:0": shape (32, 32), type "<f4">
<HDF5 group "/model_weights/dense_4" (1 members)>
<HDF5 group "/model_weights/dense_4/dense_4" (2 members)>
<HDF5 dataset "bias:0": shape (6,), type "<f4">
<HDF5 dataset "kernel:0": shape (32, 6), type "<f4">
<HDF5 group "/optimizer_weights" (2 members)>
<HDF5 group "/optimizer_weights/Adam" (1 members)>
<HDF5 dataset "iter